In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [4]:
spark

In [5]:
songPath = 'data/song_data/*/*/*/*.json'
logPath = 'data/log-data/*.json'
output_data = 'spark-warehouse/'

In [6]:
song_data = os.path.join(songPath)
    
song_df = spark.read.json(song_data)
song_df.createOrReplaceTempView("song_dfview")

songs_table = spark.sql('''
        SELECT DISTINCT
            song_id, 
            title, 
            artist_id, 
            year, 
            duration
        FROM song_dfview
        WHERE song_id IS NOT NULL''')
    
songs_table.write.partitionBy("year", "artist_id").mode('overwrite').parquet(output_data + '/songs/songs_table.parquet')

artists_table = spark.sql('''
        SELECT DISTINCT
            artist_id,
            artist_name AS name,
            artist_location AS location,
            artist_latitude AS lattitude,
            artist_longitude AS longitude
        FROM song_dfview
        WHERE artist_id IS NOT NULL''')
    
artists_table.write.mode('overwrite').parquet(output_data + '/artists/artists_table.parquet')

In [7]:
songs_table.show(5)

+------------------+--------------------+------------------+----+---------+
|           song_id|               title|         artist_id|year| duration|
+------------------+--------------------+------------------+----+---------+
|SOGNCJP12A58A80271|Do You Finally Ne...|ARB29H41187B98F0EF|1972|342.56934|
|SOOJPRH12A8C141995|   Loaded Like A Gun|ARBGXIG122988F409D|   0|173.19138|
|SOFCHDR12AB01866EF|         Living Hell|AREVWGE1187B9B890A|   0|282.43546|
|SOWTBJW12AC468AC6E|Broken-Down Merry...|ARQGYP71187FB44566|   0|151.84934|
|SOGOSOV12AF72A285E|   ¿Dónde va Chichi?|ARGUVEV1187B98BA17|1997|313.12934|
+------------------+--------------------+------------------+----+---------+
only showing top 5 rows



In [8]:
artists_table.where('lattitude IS NOT null').show(5)

+------------------+----------------+--------------------+---------+----------+
|         artist_id|            name|            location|lattitude| longitude|
+------------------+----------------+--------------------+---------+----------+
|ARPBNLO1187FB3D52F|        Tiny Tim|        New York, NY| 40.71455| -74.00712|
|AR0IAWL1187B9A96D0|    Danilo Perez|              Panama|   8.4177| -80.11278|
|ARMBR4Y1187B9990EB|    David Martin|     California - SF| 37.77916|-122.42005|
|AR0RCMP1187FB3F427|Billie Jo Spears|        Beaumont, TX| 30.08615| -94.10158|
|AROUOZZ1187B9ABE51|     Willie Bobo|New York, NY [Spa...| 40.79195| -73.94512|
+------------------+----------------+--------------------+---------+----------+
only showing top 5 rows



In [9]:
log_data = os.path.join(logPath)

log_df = spark.read.json(log_data)
    
# filter by actions for song plays
log_df = log_df.filter(log_df.page == 'NextSong')
log_df.createOrReplaceTempView("log_dfview")

users_table = spark.sql('''
        SELECT DISTINCT
            userId AS user_id,
            firstName AS first_name,
            lastName AS last_name,
            gender,
            level
        FROM log_dfview
        WHERE userId IS NOT NULL''')

users_table.write.mode('overwrite').parquet(output_data + "/users/users_table.parquet")

In [10]:
users_table.show(5)

+-------+----------+---------+------+-----+
|user_id|first_name|last_name|gender|level|
+-------+----------+---------+------+-----+
|     98|    Jordyn|   Powell|     F| free|
|     34|    Evelin|    Ayala|     F| free|
|     85|   Kinsley|    Young|     F| paid|
|     38|    Gianna|    Jones|     F| free|
|     85|   Kinsley|    Young|     F| free|
+-------+----------+---------+------+-----+
only showing top 5 rows



In [11]:
log_df.describe

<bound method DataFrame.describe of DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: double, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]>

In [13]:
from pyspark.sql import types as T

get_timestamp = udf(lambda x: datetime.fromtimestamp( (x/1000.0) ), T.TimestampType()) 
get_hour = udf(lambda x: x.hour, T.IntegerType()) 
get_day = udf(lambda x: x.day, T.IntegerType()) 
get_week = udf(lambda x: x.isocalendar()[1], T.IntegerType()) 
get_month = udf(lambda x: x.month, T.IntegerType()) 
get_year = udf(lambda x: x.year, T.IntegerType()) 
get_weekday = udf(lambda x: x.weekday(), T.IntegerType()) 


log_df = log_df.withColumn("start_time", get_timestamp(log_df.ts))
log_df = log_df.withColumn("hour", get_hour(log_df.start_time))
log_df = log_df.withColumn("day", get_day(log_df.start_time))
log_df = log_df.withColumn("week", get_week(log_df.start_time))
log_df = log_df.withColumn("month", get_month(log_df.start_time))
log_df = log_df.withColumn("year", get_year(log_df.start_time))
log_df = log_df.withColumn("weekday", get_weekday(log_df.start_time))

time_cols = ["start_time", "hour", "day", "week", "month", "year", "weekday"]
time_table = log_df.select(time_cols)    

time_table.printSchema

<bound method DataFrame.printSchema of DataFrame[start_time: timestamp, hour: int, day: int, week: int, month: int, year: int, weekday: int]>

In [14]:
type(users_table)

pyspark.sql.dataframe.DataFrame

In [21]:
from pyspark.sql import functions as F
os.environ["SPARK_HOME"] = "/usr/local/Cellar/apache-spark/1.5.1/"
os.environ["PYSPARK_PYTHON"]="/usr/local/bin/python3"

song_df.createOrReplaceTempView("songView")
log_df.createOrReplaceTempView("logView")
time_table.createOrReplaceTempView("TimeView")
        
songplays_table = spark.sql('''
        SELECT DISTINCT
            t.start_time,
            u.userId AS user_id,
            u.level AS level,
            s.song_id AS song_id,
            s.artist_id AS artist_id,
            u.sessionId AS session_id,
            s.artist_location AS location,
            u.userAgent AS user_agent,
            t.year,
            t.month
        FROM logView u
        JOIN songView s ON u.song = s.title
        JOIN TimeView t ON u.ts = CAST(t.start_time AS bigint)
        AND u.artist = s.artist_name
        AND u.length = s.duration''')

songplays_table = songplays_table.withColumn("songplay_id", F.monotonically_increasing_id())
    
songplays_table.select("songplay_id").show(5)

Py4JJavaError: An error occurred while calling o607.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(start_time#1092, user_id#1079, level#128, song_id#14, artist_id#7, session_id#1083L, location#1084, user_agent#1085, year#1087, month#1088, 200), true, [id=#982]
+- *(5) HashAggregate(keys=[start_time#1092, user_id#1079, level#128, song_id#14, artist_id#7, session_id#1083L, location#1084, user_agent#1085, year#1087, month#1088], functions=[], output=[start_time#1092, user_id#1079, level#128, song_id#14, artist_id#7, session_id#1083L, location#1084, user_agent#1085, year#1087, month#1088])
   +- *(5) Project [start_time#1092, userId#138 AS user_id#1079, level#128, song_id#14, artist_id#7, sessionId#133L AS session_id#1083L, artist_location#9 AS location#1084, userAgent#137 AS user_agent#1085, year#1087, month#1088]
      +- *(5) BroadcastHashJoin [ts#136L], [cast(start_time#1092 as bigint)], Inner, BuildRight
         